In [1]:
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pgbm_nb
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split

from src.probabilistic_flow_boosting.pipelines.modeling.utils import setup_random_seed

RANDOM_SEED = 1

setup_random_seed(RANDOM_SEED)

In [2]:
df = pd.read_csv('data/01_raw/CatData/avocado/avocado.csv', index_col=0)

In [3]:
x = df.drop(columns = ['Date', 'AveragePrice'])
x = pd.get_dummies(x)
y = df[['AveragePrice']]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=RANDOM_SEED)
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=RANDOM_SEED)

In [5]:
x_train.shape, x_test.shape

((14599, 65), (3650, 65))

In [6]:
def mseloss_objective(yhat, y, sample_weight=None):
    gradient = (yhat - y)
    hessian = np.ones_like(yhat)
    return gradient, hessian


def rmseloss_metric(yhat, y, sample_weight=None):
    loss = np.sqrt(np.mean(np.square(yhat - y)))
    return loss

In [7]:
model = pgbm_nb.PGBM()

params = {
    'min_split_gain':0,
    'min_data_in_leaf':2,
    'max_leaves':8,
    'max_bin':64,
    'learning_rate':0.1,
    'verbose':2,
    'early_stopping_rounds':200,
    'feature_fraction':1,
    'bagging_fraction':1,
    'seed':RANDOM_SEED,
    'reg_lambda':1,
    'device':'gpu',
    'gpu_device_id':0,
    'derivatives':'exact',
    'distribution':'normal',
    'n_estimators': 2000
}

In [8]:
model.train(
    train_set=(x_train.values, y_train.values), 
    objective=mseloss_objective, 
    metric=rmseloss_metric, 
    valid_set=(x_val.values, y_val.values),
    params=params
)

Estimator 0/2000, Train metric: 0.3857, Validation metric: 0.3950
Estimator 1/2000, Train metric: 0.3711, Validation metric: 0.3802
Estimator 2/2000, Train metric: 0.3588, Validation metric: 0.3681
Estimator 3/2000, Train metric: 0.3479, Validation metric: 0.3570
Estimator 4/2000, Train metric: 0.3383, Validation metric: 0.3475
Estimator 5/2000, Train metric: 0.3306, Validation metric: 0.3396
Estimator 6/2000, Train metric: 0.3236, Validation metric: 0.3324
Estimator 7/2000, Train metric: 0.3177, Validation metric: 0.3265
Estimator 8/2000, Train metric: 0.3124, Validation metric: 0.3212
Estimator 9/2000, Train metric: 0.3077, Validation metric: 0.3165
Estimator 10/2000, Train metric: 0.3039, Validation metric: 0.3126
Estimator 11/2000, Train metric: 0.3006, Validation metric: 0.3093
Estimator 12/2000, Train metric: 0.2978, Validation metric: 0.3066
Estimator 13/2000, Train metric: 0.2949, Validation metric: 0.3036
Estimator 14/2000, Train metric: 0.2921, Validation metric: 0.3010
Estim

In [9]:
model.optimize_distribution(x_val.values, y_val.values.reshape(-1))

CRPS: 0.22 (Distribution: normal, Tree correlation: 0.000)
CRPS: 0.17 (Distribution: normal, Tree correlation: 0.010)
CRPS: 0.13 (Distribution: normal, Tree correlation: 0.020)
CRPS: 0.11 (Distribution: normal, Tree correlation: 0.030)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.040)
CRPS: 0.09 (Distribution: normal, Tree correlation: 0.050)
CRPS: 0.09 (Distribution: normal, Tree correlation: 0.060)
CRPS: 0.09 (Distribution: normal, Tree correlation: 0.070)
CRPS: 0.09 (Distribution: normal, Tree correlation: 0.080)
CRPS: 0.09 (Distribution: normal, Tree correlation: 0.090)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.100)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.110)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.120)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.130)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.140)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.150)
CRPS: 0.10 (Distribution: normal, Tree correlation: 0.16

('studentt', 0.04)

In [10]:
y_train_dist = model.predict_dist(x_train.values, n_forecasts=1000)
y_val_dist = model.predict_dist(x_val.values, n_forecasts=1000)
y_test_dist = model.predict_dist(x_test.values, n_forecasts=1000)

In [11]:
print(model.crps_ensemble(y_train_dist, y_train.values.reshape(-1)).mean())
print(model.crps_ensemble(y_val_dist, y_val.values.reshape(-1)).mean())
print(model.crps_ensemble(y_test_dist, y_test.values.reshape(-1)).mean())

0.08975179917251275
0.09082991893713253
0.10305392201337968
